In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\3\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\3\test'
checkpoint_path = r'C:\Users\24306\Desktop\EfficientNetV2B0\3\EfficientNetV2B0.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[40.0, 20.0, 66.0, 37.0, 95.0, 40.0, 80.0, 39.0, 47.0, 16.0, 80.0, 47.0, 86.0, 20.0, 10.0, 47.0, 47.0, 93.0, 47.0, 93.0, 35.0, 7.0, 100.0, 16.0, 45.0, 96.0, 10.0, 96.0, 23.0, 20.0, 47.0, 20.0, 80.0, 37.0, 93.0, 66.0, 84.0, 13.0, 96.0, 40.0, 30.0, 96.0, 96.0, 100.0, 95.0, 47.0, 96.0, 7.0, 35.0, 93.0, 45.0, 30.0, 66.0, 100.0, 52.0, 95.0, 13.0, 37.0, 7.0, 100.0, 13.0, 47.0, 96.0, 86.0, 95.0, 10.0, 40.0, 45.0, 47.0, 7.0, 45.0, 84.0, 84.0, 16.0, 80.0, 37.0, 7.0, 80.0, 16.0, 20.0, 86.0, 37.0, 47.0, 96.0, 16.0, 45.0, 84.0, 30.0, 93.0, 16.0, 93.0, 23.0, 47.0, 7.0, 84.0, 96.0, 32.0, 95.0, 39.0, 100.0, 10.0, 96.0, 52.0, 7.0, 96.0, 95.0, 66.0, 66.0, 4.0, 40.0, 37.0, 66.0, 66.0, 4.0, 47.0, 7.0, 30.0, 47.0, 47.0, 80.0, 86.0, 7.0, 47.0, 4.0, 96.0, 93.0, 16.0, 84.0, 37.0, 32.0, 80.0, 10.0, 80.0, 23.0, 7.0, 40.0, 45.0, 30.0, 37.0, 100.0, 37.0, 52.0, 37.0, 84.0, 4.0, 96.0, 66.0, 16.0, 10.0, 20.0, 23.0, 4.0, 20.0, 32.0, 96.0, 95.0, 93.0, 16.0, 95.0, 45.0, 47.0, 93.0, 100.0, 39.0, 95.0, 86.0, 32.0, 35.0,

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
EfficientNetV2B0 = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
EfficientNetV2B0.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = EfficientNetV2B0(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-b0 (Function  (None, 7, 7, 1280)       5919312   
 al)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 1281      
                                                                 
Total params: 5,920,593
Trainable params: 5,859,985
Non-trainable params: 60,608
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
#要改
np.savetxt('EfficientNetV2B0_loss_3.txt',history.history['loss'])
#要改
np.savetxt('EfficientNetV2B0_valloss_3.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Wed Apr  5 19:41:35 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 224.7469

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2B0\3\EfficientNetV2B0.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2B0\3\EfficientNetV2B0.ckpt\assets


591/591 [==============================] - 109s 158ms/step - loss: 224.7469 - val_loss: 488.8898
Epoch 2/300
591/591 [==============================] - 56s 94ms/step - loss: 8.5438 - val_loss: 640.7505
Epoch 3/300
591/591 [==============================] - ETA: 0s - loss: 4.3306

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2B0\3\EfficientNetV2B0.ckpt\assets


...

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2B0\3\EfficientNetV2B0.ckpt\assets


591/591 [==============================] - 130s 219ms/step - loss: 0.2247 - val_loss: 0.0709
Epoch 182/300
591/591 [==============================] - 84s 141ms/step - loss: 0.1791 - val_loss: 0.0915
Epoch 183/300
591/591 [==============================] - 85s 143ms/step - loss: 0.1892 - val_loss: 0.0971
Epoch 184/300
591/591 [==============================] - 82s 140ms/step - loss: 0.2144 - val_loss: 0.1029
Epoch 185/300
591/591 [==============================] - 84s 142ms/step - loss: 0.1782 - val_loss: 0.1160
Epoch 186/300
591/591 [==============================] - 84s 142ms/step - loss: 0.2370 - val_loss: 0.1257
Epoch 187/300
591/591 [==============================] - 84s 142ms/step - loss: 0.1989 - val_loss: 0.0849
Epoch 188/300
591/591 [==============================] - 86s 145ms/step - loss: 0.1789 - val_loss: 0.0929
Epoch 189/300
591/591 [==============================] - 84s 142ms/step - loss: 0.1811 - val_loss: 0.1212
Epoch 190/300
591/591 [==============================] - 85

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2B0\3\EfficientNetV2B0.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2B0\3\EfficientNetV2B0.ckpt\assets


591/591 [==============================] - 134s 226ms/step - loss: 0.1668 - val_loss: 0.0658
Epoch 267/300
591/591 [==============================] - 95s 161ms/step - loss: 0.1560 - val_loss: 0.0833
Epoch 268/300
591/591 [==============================] - 94s 158ms/step - loss: 0.1861 - val_loss: 0.0769
Epoch 269/300
591/591 [==============================] - 94s 159ms/step - loss: 0.1666 - val_loss: 0.0948
Epoch 270/300
591/591 [==============================] - 93s 158ms/step - loss: 0.2571 - val_loss: 0.0773
Epoch 271/300
591/591 [==============================] - 93s 157ms/step - loss: 0.1747 - val_loss: 0.0703
Epoch 272/300
591/591 [==============================] - 93s 158ms/step - loss: 0.1571 - val_loss: 0.0859
Epoch 273/300
591/591 [==============================] - 94s 159ms/step - loss: 0.1901 - val_loss: 0.0928
Epoch 274/300
591/591 [==============================] - 94s 159ms/step - loss: 0.1574 - val_loss: 0.0756
Epoch 275/300
591/591 [==============================] - 94